In [1]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('classic')

# 단순한 계층 구현하기

## 곱셈 계층

In [2]:
class MulLayer :
    def __init__(self) :
        self.x = None
        self.y = None
        
    def forward(self, x, y) :
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    def backward(self, dout) :
        dx = dout * self.y
        dy = dout * self.x
        
        return dx, dy

In [3]:
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [4]:
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


## 덧셈 계층

In [5]:
class AddLayer :
    def __init__(self) :
        pass
    
    def forward(self, x, y) :
        out = x + y
        return out
    
    def backward(self, dout) :
        dx = dout * 1
        dy = dout * 1
        return dx, dy 

### 사과 2개, 귤 3개를 사서 소비세를 더한 값을 구하는 계층

In [6]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

# 역전파
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


# ReLU 계층

In [8]:
class Relu :
    def __init__(self) :
        self.mask = None
        
    def forward(self, x) :
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout) :
        dout[self.mask] = 0
        dx = dout
        
        return dx

# Sigmoid 계층

In [ ]:
class Sigmoid :
    def __init__(self) :
        self.out = None
        
    def forward(self, x) :
        out = 1 / (1 + np.exp(-x))
        self.out = out
        
        return out
    
    def backward(self, dout) :
        dx = dout * (1.0 - self.out) * self.out
        
        return dx

# Affine 계층

In [ ]:
class Affine :
    def __init__(self, W, b) :
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x) :
        self.x = x
        out = np.dot(x, self.W) + self.b
        
        return out
    
    def backward(self, dout) :
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        return dx

In [16]:
from audioop import cross


def cross_entropy_error(y, t) :
    if y.ndim == 1 :
        t = t.reshape(1, -1)
        y = y.reshape(1, -1)
        
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size

def softmax(a) :
    c = np.max(a)
    exp_a = np.exp(a-c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

class SoftmaxWithLoss :
    def __init__(self) :
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x, t) :
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1) :
        # 역전파시에는 batch 수로 나눠야 데이터 1개당 오차를 앞 계층으로 전파한다.
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx

# 오차역전파법을 적용한 신경망 구현하기